In [1]:
import sys

sys.path.insert(0, '..')

import os
import requests
import json

In [2]:
from biosimulator_processes.compare import ComparisonDocument
from biosimulator_processes import CORE
from process_bigraph import pp, Composite

### Example Composite process using copasi and tellurium

In [3]:
simulators = ['copasi', 'tellurium']  # , 'amici']
duration = 30
n_steps = 200
model_filepath = '../biosimulator_processes/model_files/sbml/Caravagna2010.xml'


comparison = ComparisonDocument(simulators, duration, n_steps, model_filepath)

In [4]:
pp(comparison.composite)

### ODE Simulator Comparison

In [5]:
from biosimulator_processes.compare import ComparisonDocument

sbml_model_path = '../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml'
simulators = ['copasi', 'copasi']  # , 'tellurium']
duration = 30 
n_steps = 50 
target_parameter = {
    'name': 'plasmin',
    'value': 2.01
}


def create_comparison_document(
        sbml_model_path: str, 
        simulators: list[str], 
        duration: int, 
        n_steps: int, 
        target_param: dict = None
        ) -> ComparisonDocument:
    return ComparisonDocument(
    simulators=simulators, 
    duration=duration, 
    num_steps=n_steps,
    model_filepath=sbml_model_path,
    target_parameter=target_param)


def generate_workflow(document: ComparisonDocument):
    return Composite(
        config={'state': document.composite},
        core=CORE)


ode_comparison_document = create_comparison_document(sbml_model_path, simulators, duration, n_steps, target_parameter)

In [6]:
pp(ode_comparison_document.composite)

In [7]:
ode_comparison_workflow = Composite(config={'state': ode_comparison_document.composite}, core=CORE)

In [8]:
ode_comparison_workflow.run(duration)
ode_comparison_results = ode_comparison_workflow.gather_results()

In [9]:
ode_comparison_results

In [10]:
import tellurium as te

simulator = te.loadSBMLModel('../biosimulator_processes/model_files/sbml/BIOMD0000000630_url.xml')


In [11]:
dir(simulator)

In [12]:
simulator.getReactionRates()

In [13]:
simulator.getReactionIds()

In [14]:
simulator.integrator = 'stochastic'

In [ ]:
simulator.integrator

In [ ]:
simulator.getIds()

In [6]:
import numpy as np 
type(np.random.rand(3, 4))

In [2]:
import numpy as np
from dataclasses import dataclass
from random import randint


@dataclass
class BestOutput:
    value: float 
    generation: int 
    index: int 
    winning_value: float 
    

# Fetch experimental data
def get_experimental_data(param_name: str = None) -> float:
    # TODO: enable this
    return np.random.rand()

# Initialize parameters
winning_value = get_experimental_data()  # TODO: Fetch this data from experimental online
population_size = 100  # Number of simulators being compared
mutation_rate = 0.1  # Initial mutation rate
range_low = 0  
range_high = 20  # TODO: make low and high derived automatically from population size and winning value


# Fitness function
def calculate_fitness(population):
    return [abs(individual - winning_value) for individual in population]

# Selection function
def select_individuals(fitness, num_parents):
    fitness = np.array(fitness)
    sorted_indices = np.argsort(fitness)
    return sorted_indices[:num_parents]

# Crossover function
def crossover(parents, num_offspring):
    offspring = []
    for _ in range(num_offspring):
        parent1, parent2 = np.random.choice(parents, 2, replace=False)
        child = (parent1 + parent2) / 2
        offspring.append(child)
    return offspring

# Mutation function
def mutate(offspring, mutation_rate):
    for i in range(len(offspring)):
        if np.random.rand() < mutation_rate:
            offspring[i] += np.random.normal(0, 0.1)
    return offspring

# Feedback function to adjust algorithm parameters based on real-world testing
def feedback_adjustment(best_output, current_range):
    error = abs(best_output - winning_value)
    new_mutation_rate = max(0.01, mutation_rate - 0.02 * (1 - error / (range_high - range_low)))
    new_range_low, new_range_high = current_range
    if error < 1:
        new_range_low, new_range_high = best_output - 5, best_output + 5
    return new_mutation_rate, new_range_low, new_range_high

# Main genetic algorithm function with dynamic stopping
def genetic_algorithm(improvement_threshold=0.000001, max_stagnation=40):
    global range_low, range_high, mutation_rate
    population = np.random.uniform(low=range_low, high=range_high, size=population_size)
    best_output = None
    best_index = None
    last_best_fitness = float('inf')
    stagnation_counter = 0

    generation = 0
    while True:
        fitness = calculate_fitness(population)
        current_best_fitness = min(fitness)
        print(f"Generation {generation}")
        print("Best fitness:", current_best_fitness)

        if current_best_fitness < last_best_fitness:
            if last_best_fitness - current_best_fitness < improvement_threshold:
                stagnation_counter += 1
            else:
                stagnation_counter = 0
            last_best_fitness = current_best_fitness
        else:
            stagnation_counter += 1

        if stagnation_counter >= max_stagnation:
            print(f"Stopping: No improvement in {max_stagnation} generations.")
            break

        selected_indices = select_individuals(fitness, population_size // 2)
        selected = [population[i] for i in selected_indices]

        offspring = crossover(selected, population_size - len(selected))
        offspring = mutate(offspring, mutation_rate)

        population = np.array(selected + offspring)

        best_index = np.argmin(calculate_fitness(population))
        best_output = population[best_index]
        mutation_rate, range_low, range_high = feedback_adjustment(best_output, (range_low, range_high))

        generation += 1

    return BestOutput(value=best_output, generation=generation, index=best_index, winning_value=winning_value)


# Running the enhanced genetic algorithm
best_output = genetic_algorithm()
print(f"The winning value: {winning_value}")
print("Best output closest to winning value with dynamic stopping:", best_output)